In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
import VertRegridFlexLL as Vrg
import PlotUtil as Pu
import GridUtils as Gu
import validation_data as Val

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( Pu )
importlib.reload( Gu )
importlib.reload(Av)
importlib.reload(Val)


In [ ]:
#####
exp_Val='ERA-5'

user_C,user_X='juliob','juliob'

#subd='hist'
subd='regridded'

"""
exp_C = 'test3.01'
exp_X = 'test3_gw_spread.02'
"""

exp_C = 'test5_NO_spread'
#exp_X = 'test5_gw_spread.02.2'  #'test3.01'

#.     fmt_N_rd1p0_zm4x
#exp_C='fmt_N_rd1p0_zm4x'
exp_C='fmt_N_rd1p0_zm4x_r02'
exp_X='fmtndg_N_x03'

#exp_X = 'test4_gw_spread.02.2'
#exp_C = 'test3.02.2'


#exp_C = 'test3.02.2'
#exp_C = 'test5_gw_spread.02.2'  #'test3.01'
#exp_X = 'test5_NO_spread'  #'test3.01'
#exp_X = 'test3.02.2'|


"""
user_C='tilmes'
#user_X='tilmes'
exp_C ='f.cam6_3_132.FMTHIST_ne30.aircraft'
#exp_X ='f.cam6_3_132.FMTHIST_ne30.rdg_cd_llb_01'
"""


In [ ]:
###

if (user_C == 'juliob' ):
    path_C = '/glade/derecho/scratch/juliob/archive/'+exp_C+'/atm/'+ subd +'/'+exp_C+'.cam.h0.*.nc' 
elif (user_C == 'tilmes' ):
    path_C = '/glade/derecho/scratch/tilmes/archive/'+exp_C+'/atm/hist/'+exp_C+'.cam.h0.*.nc' 


if (user_X == 'juliob' ):
    path_X = '/glade/derecho/scratch/juliob/archive/'+exp_X+'/atm/'+ subd +'/'+exp_X+'.cam.h0.*.nc' 
elif (user_X == 'tilmes' ):
    path_X = '/glade/derecho/scratch/tilmes/archive/'+exp_X+'/atm/hist/'+exp_X+'.cam.h0.*.nc' 
  
    


In [ ]:
print(path_C)

In [ ]:
###

Dc = xr.open_mfdataset( path_C ,data_vars='different', coords='different' )
Dx = xr.open_mfdataset( path_X ,data_vars='different', coords='different' )

#D2b = xr.open_mfdataset( '/glade/derecho/scratch/juliob/archive/test02/atm/hist/test02.cam.h0.*.nc' , combine='by_coords', data_vars='minimal', coords='minimal')
#D2c = xr.open_mfdataset( '/glade/derecho/scratch/juliob/archive/test02/atm/hist/test02.cam.h0.*.nc' , combine='by_coords', join='exact' )
#D2c = xr.open_mfdataset( '/glade/derecho/scratch/juliob/archive/test02/atm/hist/test02.cam.h0.*.nc' , combine='by_coords', coords='minimal')

#concat_dim=None )

In [ ]:
print( Gu.gridKey(Dx.lon))

In [ ]:
nyr_max=1000


nyr=np.minimum( nyr_max , 12*(len(Dc.time)//12) )
print(f"Discarding last {len(Dc.time)-nyr} months of {exp_C}" )
Dc=Dc.isel(time=np.arange(nyr))


nyr=np.minimum( nyr_max , 12*(len(Dx.time)//12) )
print(f"Discarding last {len(Dx.time)-nyr} months of {exp_X}" )
Dx=Dx.isel(time=np.arange(nyr))


In [ ]:
##
ps=Dc.PS.values
lon=Dc.lon.values
lat=Dc.lat.values
lev=Dc.lev.values


In [ ]:
print(lev[93-45])
zlev=-7.0*np.log( lev/1_000. )
print( zlev[93-45] )

In [ ]:
season='nov'
PRCc_s = Av.Seasonal( ds=Dc, season=season , fld='PRECT', dims='tyx')
PRCx_s = Av.Seasonal( ds=Dx, season=season , fld='PRECT', dims='tyx')

In [ ]:
plt.contour(lon,lat,PRCx_s)

In [ ]:
season='jun'
UUc_sz = Av.SeasonalZonal( ds=Dc, season=season , fld='U')
UUx_sz = Av.SeasonalZonal( ds=Dx, season=season , fld='U')

In [ ]:
UUs_val,zlev_val,lat_val,lon_val = Val.data(fld='U',season=season,zlev=True, Years='2000' )

In [ ]:
###


UUsz_val=np.average( UUs_val , axis=2 )
UUsz_val.shape

In [ ]:
fig = plt.figure(figsize=(20, 5))
clev=np.linspace( -60,140,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

Axes1 = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat,zlev,UUc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat,zlev,UUc_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{exp_C}> {season.upper()}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat,zlev,UUx_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat,zlev,UUx_sz ,levels=clev, colors='black')
ax1.set_title(f"Test <{exp_X}> {season.upper()}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_val,zlev_val,UUsz_val ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_val,zlev_val,UUsz_val ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

"""

Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat,zlev,UUx_sz-UUc_sz ,levels=dlev, cmap='bwr' )
co2=ax1.contour(lat,zlev,UUx_sz-UUc_sz ,levels=dlev, colors='black')
ax1.set_title('Diff '+season.upper()  )
cb=plt.colorbar(co1)
"""


In [ ]:
########
#season='djf'
#UUc_s = Av.Seasonal( ds=Dc, season=season , fld='U')
NtUx_s = Av.Seasonal( ds=Dx, season=season , fld='Nudge_U')

In [ ]:
NtUx_s.shape
NtUx_sz = np.average( NtUx_s  , axis=2 )

In [ ]:
print(zlev[10],zlev_val[14])
print(zlev[55],zlev_val[14])
print(zlev[83],zlev_val[32])


In [ ]:
fig = plt.figure(figsize=(15, 5))
klev=10
clev=5*np.linspace( -2,2,num=21) 
Axes1 = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )


co=ax1.contourf(lon,lat,86400.*NtUx_s[klev,:,:] ,levels=clev, cmap='bwr' )
pr1=ax1.contour( lon,lat,ps[0,:,:],levels=[10_000,95_000,100_100],colors='black')
ax1.set_title(f'{season.upper()} Nudge_U at Z={zlev[klev]:.2f} km')
plt.colorbar(co)

In [ ]:
fig = plt.figure(figsize=(15, 5))
klev=7
clev=5*np.linspace( -2,2,num=21) 
Axes1 = Pu.axes_def(n=1,nxplo=1,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )


co=ax1.contourf(lon,lat,86400.*NtUx_s[klev,:,:] ,levels=clev, cmap='bwr' )
pr1=ax1.contour( lon,lat,ps[0,:,:],levels=[10_000,95_000,100_100],colors='black')
ax1.set_title(f'{season.upper()} Nudge_U at Z={zlev[klev]:.2f} km')
plt.colorbar(co)

In [ ]:
clev=np.array([-20,-10,-5,-2,-1,-.5,-.2,-.1,0,.1,.2,.5,1,2,5,10,20 ] )  #np.linspace( -5,5,num=21) 
cmapx=Pu.step_cmap(cmapN='bwr',levels=clev)
plt.contourf(lat,zlev,86400.*NtUx_sz ,levels=clev, cmap='bwr' )
plt.title("Zonal mean Nudge_U SON")
plt.colorbar()

In [ ]:
fig = plt.figure(figsize=(15, 5))
clev=np.linspace(0,10,num=16)
dlev=np.linspace(-1,1,num=11) * 10

clev=np.linspace(-50,60,num=47)
clev=np.linspace(-50,60,num=22+1)
klev=55
klev_val=14

Axes1 = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon,lat,UUc_s[klev,:,:]  ,levels=clev , cmap='gist_ncar' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Control')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon,lat,UUx_s[klev,:,:]  ,levels=clev , cmap='gist_ncar' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Control')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon_val,lat_val,UUs_val[klev_val,:,:]  ,levels=clev , cmap='gist_ncar' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Control')
cb=plt.colorbar(co1)

"""
Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon,lat,u10_x.mean(axis=0)  ,levels=clev, cmap='gist_ncar' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Test')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon,lat,u10_x.mean(axis=0)-u10_c.mean(axis=0)  ,levels=dlev, cmap='bwr' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Test - Control')
cb=plt.colorbar(co1)
"""


In [ ]:
fig = plt.figure(figsize=(15, 5))
clev=np.linspace(0,10,num=16)
dlev=np.linspace(-1,1,num=11) * 10

clev=np.linspace(-50,60,num=47)
clev=np.linspace(-50,60,num=22+1)
klev=83
klev_val=32

Axes1 = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon,lat,UUc_s[klev,:,:]  ,levels=clev , cmap='gist_ncar' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Control')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon,lat,UUx_s[klev,:,:]  ,levels=clev , cmap='gist_ncar' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Control')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon_val,lat_val,UUs_val[klev_val,:,:]  ,levels=clev , cmap='gist_ncar' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Control')
cb=plt.colorbar(co1)

"""
Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon,lat,u10_x.mean(axis=0)  ,levels=clev, cmap='gist_ncar' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Test')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.contourf( lon,lat,u10_x.mean(axis=0)-u10_c.mean(axis=0)  ,levels=dlev, cmap='bwr' ) 
pr1 = ax1.contour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Test - Control')
cb=plt.colorbar(co1)
"""


In [ ]:
plt.contourf(lat,zlev,UUc_s ,levels=clev )
plt.colorbar()

In [ ]:
print(np.shape(u10_c))
print(np.shape(u10_x))
print(np.shape(lon))
print(np.shape(lat))

print(Dc.lev.values[20])

In [ ]:
fig = plt.figure(figsize=(15, 5))
clev=np.linspace(0,10,num=16)
dlev=np.linspace(-1,1,num=11) * 10

clev=np.linspace(-20,50,num=16)


Axes1 = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.tricontourf( lon,lat,u10_c.mean(axis=0)  ,levels=clev , cmap='gist_ncar' ) 
pr1 = ax1.tricontour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Control')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.tricontourf( lon,lat,u10_x.mean(axis=0)  ,levels=clev, cmap='gist_ncar' ) 
pr1 = ax1.tricontour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Test')
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1 = ax1.tricontourf( lon,lat,u10_x.mean(axis=0)-u10_c.mean(axis=0)  ,levels=dlev, cmap='bwr' ) 
pr1 = ax1.tricontour( lon,lat,ps[0,:],levels=[10_000,100_100],colors='black')
ax1.set_title('Test - Control')
cb=plt.colorbar(co1)
